# Most liveable and affortable suburb
This notebook is used to find how liveable and affortable each suburb is 

In [ ]:
## This note book is used to find how liveable and affortable each suburb is 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
## Read in the final domain csv
df = pd.read_csv("../data/curated/domain_final.csv")

In [ ]:
df.head()

In [ ]:
## To calculated affortable, we will use the formular (average cost in suburb)/(average income in suburb)
## create a new dara frame on contain postcode, cost and average income

affortable_df = pd.DataFrame(df,columns=["postcode","cost","avg_income"])

In [ ]:
## calculate the ratio of property's weekly cost and the average income in suburb

affortable_df["cost_gap"]=affortable_df["cost"]/affortable_df["avg_income"]

In [ ]:
affortable_df.head()


In [ ]:
affortable_df.shape

In [ ]:
## Find null and delete it 
affortable_df.isnull().sum()

In [ ]:
## drop all null before we calculate the average

affortable_df.dropna(subset=['avg_income'], inplace=True)

In [ ]:
## Group all property in the same suberb and find the mean ratio

cost_gap_output= affortable_df.groupby("postcode").agg({'cost_gap':'mean'})


In [ ]:
cost_gap_output.head()

In [ ]:
## Get the top 10 affortable suburb(most affortable)

print(cost_gap_output.sort_values(by=["cost_gap"],ascending=True)[:10])

In [ ]:
## Get the top 10  not affortable suburb(most not affortable)

print(cost_gap_output.sort_values(by=["cost_gap"],ascending=False)[:10])

In [ ]:
## Read in the csv that record all the hospital in victoria

df_2 = pd.read_csv("../data/curated/MainHospitalList (1).csv")

In [ ]:
df_2.head()

In [ ]:
## Count the number of hospital in each suberb

hospital_count = df_2.loc[:,"Postcode"].value_counts().rename_axis('postcode').reset_index(name='hos_count')

In [ ]:
hospital_count.head()

In [ ]:
## Read in the csv that record all the school in victoria
df_3 = pd.read_csv("../data/curated/dv309_schoollocations2021.csv", encoding='cp1252')

In [ ]:
df_3.head()

In [ ]:
## Count the number of school in each suberb
school_count = df_3.loc[:,"Postal_Postcode"].value_counts().rename_axis('postcode').reset_index(name='school_count')

In [ ]:
school_count.head()

In [ ]:
## Create two data frame which will be used to count the average distance to bus stop and train station
## the distance is average in each suburb

bus_df = pd.DataFrame(df,columns=["postcode","dist_nearest_busstop"])
train_df=pd.DataFrame(df,columns=["postcode","dist_nearest_trainstation"])

In [ ]:
## Find null 

bus_df.isnull().sum()

In [ ]:
## Find null 

train_df.isnull().sum()

In [ ]:
## calculate the average distance to the bus stop and train station in each suburb

bus_output= bus_df.groupby("postcode").agg({'dist_nearest_busstop':'mean'}).reset_index()
train_output = train_df.groupby("postcode").agg({'dist_nearest_trainstation':'mean'}).reset_index()

In [ ]:
bus_output.head()

In [ ]:
train_output.head()

In [ ]:
## Creat a data frame contains all post code that we have domain data with

rank_df=pd.DataFrame(df['postcode'].unique(), columns= ['postcode'] )
rank_df

In [ ]:
## adding numer of school, number of hospital, distance to train station and distance to bus stop to the result data frame

rank_df=rank_df.join(train_output.set_index("postcode"),on="postcode")
rank_df=rank_df.join(bus_output.set_index("postcode"),on="postcode")
rank_df=rank_df.join(school_count.set_index("postcode"),on="postcode")
rank_df=rank_df.join(hospital_count.set_index("postcode"),on="postcode")

In [ ]:
## if null, add 0 in to finish calculation

rank_df['school_count'].fillna(0, inplace=True)
rank_df['hos_count'].fillna(0, inplace=True)

In [ ]:
rank_df.head()

In [ ]:
rank_df.shape

In [ ]:
## normalization

x = rank_df[['dist_nearest_trainstation', 'dist_nearest_busstop','school_count','hos_count']].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
rank_final1 = pd.DataFrame(x_scaled, columns= ['dist_nearest_trainstation', 'dist_nearest_busstop','school_count','hos_count'] )

In [ ]:
rank_df['dist_nearest_trainstation'] = rank_final1['dist_nearest_trainstation']
rank_df['dist_nearest_busstop'] = rank_final1['dist_nearest_busstop']
rank_df['school_count'] = rank_final1['school_count']
rank_df['hos_count'] = rank_final1['hos_count']

In [ ]:
## change the distance to negative
## we want suburb with small distance have higher score

rank_df["dist_nearest_trainstation"]=rank_df["dist_nearest_trainstation"]*-1
rank_df["dist_nearest_busstop"]=rank_df["dist_nearest_busstop"]*-1

In [ ]:
rank_df.head()

In [ ]:
## calculate final rank
## scale the distance up and scale bus stop and train staion down

rank_df['rank'] =  100*rank_df.dist_nearest_trainstation + 100*rank_df.dist_nearest_busstop + 0.5*rank_df.school_count + 0.5*rank_df.hos_count

In [ ]:
rank_df.head()

In [ ]:
## Calculate the top 10 liveable suburb

top_10 = rank_df.sort_values('rank',ascending=False).head(11)
top_10